<a href="https://colab.research.google.com/github/akshhj/nlp-lab/blob/main/NLP_EX8_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# viterbi

In [ ]:
import numpy as np

# Define the HMM components
states = ['DT', 'NN', 'VB']  # Tags
observations = ['the', 'dog', 'runs']  # Sentence

# Initial probabilities
initial_probs = {'DT': 0.8, 'NN': 0.15, 'VB': 0.05}

# Transition probabilities
transition_probs = {
    'DT': {'DT': 0.05, 'NN': 0.9, 'VB': 0.05},
    'NN': {'DT': 0.05, 'NN': 0.15, 'VB': 0.8},
    'VB': {'DT': 0.3, 'NN': 0.4, 'VB': 0.3}
}

# Emission probabilities
emission_probs = {
    'DT': {'the': 0.95, 'dog': 0.02, 'runs': 0.01},
    'NN': {'the': 0.01, 'dog': 0.9, 'runs': 0.1},
    'VB': {'the': 0.01, 'dog': 0.03, 'runs': 0.85}
}

# Viterbi Algorithm
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]  # Viterbi table
    path = {}  # To track the best path

    # Initialize base cases (t = 0)
    for state in states:
        V[0][state] = start_p[state] * emit_p[state][obs[0]]
        path[state] = [state]

    # Run Viterbi for t > 0
    for t in range(1, len(obs)):
        V.append({})
        new_path = {}
        for curr_state in states:
            # Find the max probability path to current state
            (max_prob, best_prev_state) = max(
                (V[t-1][prev_state] * trans_p[prev_state][curr_state] * emit_p[curr_state][obs[t]], prev_state)
                for prev_state in states
            )
            V[t][curr_state] = max_prob
            new_path[curr_state] = path[best_prev_state] + [curr_state]
        path = new_path

    # Find the final state with max probability
    (max_prob, final_state) = max((V[-1][state], state) for state in states)
    return (max_prob, path[final_state])

# Run the algorithm
prob, best_path = viterbi(observations, states, initial_probs, transition_probs, emission_probs)

# Output results
print(f"Sentence: {' '.join(observations)}")
print(f"Best POS tag sequence: {' -> '.join(best_path)}")
print(f"Probability: {prob:.6f}")

Sentence: the dog runs
Best POS tag sequence: DT -> NN -> VB
Probability: 0.418608


In [ ]:
import nltk
from collections import defaultdict

def calculate_hmm_probabilities(tagged_sentences):
    """
    Calculates initial, transition, and emission probabilities from tagged sentences.

    Args:
        tagged_sentences: A list of sentences, where each sentence is a list of (word, tag) tuples.

    Returns:
        initial_probabilities: A dictionary of initial state probabilities.
        transition_probabilities: A dictionary of transition probabilities.
        emission_probabilities: A dictionary of emission probabilities.
        states: a set of all states.
    """

    initial_counts = defaultdict(int)
    transition_counts = defaultdict(lambda: defaultdict(int))
    emission_counts = defaultdict(lambda: defaultdict(int))
    state_counts = defaultdict(int)

    for sentence in tagged_sentences:
        if not sentence:
            continue #skip empty sentences
        initial_counts[sentence[0][1]] += 1
        state_counts[sentence[0][1]] += 1

        for i in range(len(sentence) - 1):
            current_tag = sentence[i][1]
            next_tag = sentence[i + 1][1]
            word = sentence[i][0]
            transition_counts[current_tag][next_tag] += 1
            emission_counts[current_tag][word] += 1
            state_counts[next_tag]+=1
        emission_counts[sentence[-1][1]][sentence[-1][0]] +=1

    states = set(state_counts.keys())
    # Calculate initial probabilities
    total_initial = sum(initial_counts.values())
    initial_probabilities = {tag: count / total_initial for tag, count in initial_counts.items()}

    # Calculate transition probabilities
    transition_probabilities = {}
    for prev_tag, next_tags in transition_counts.items():
        total_transitions = sum(next_tags.values())
        transition_probabilities[prev_tag] = {tag: count / total_transitions for tag, count in next_tags.items()}
        for state in states:
          if state not in transition_probabilities[prev_tag]:
            transition_probabilities[prev_tag][state] = 0

    # Calculate emission probabilities
    emission_probabilities = {}
    for tag, words in emission_counts.items():
        total_emissions = sum(words.values())
        emission_probabilities[tag] = {word: count / total_emissions for word, count in words.items()}

    return initial_probabilities, transition_probabilities, emission_probabilities, states

# Example usage with NLTK's treebank corpus
nltk.download('treebank')
nltk.download('universal_tagset')

tagged_sentences = nltk.corpus.treebank.tagged_sents(tagset='universal')

initial_probs, transition_probs, emission_probs, states = calculate_hmm_probabilities(tagged_sentences)

# Example output (showing a few probabilities)
print("Initial probabilities:")
print(dict(list(initial_probs.items())[:5]))  # Print a few initial probabilities

print("\nTransition probabilities:")
print(dict(list(transition_probs.items())[:2])) # Print a few transition probabilities

print("\nEmission probabilities:")
print(dict(list(emission_probs.items())[:2])) #print a few emission probabilities

print("\nStates:")
print(states)

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


Initial probabilities:
{'NOUN': 0.29151762902401634, 'DET': 0.23684210526315788, 'ADP': 0.1290240163515585, 'PRON': 0.07358201328564129, '.': 0.08175779253960143}

Transition probabilities:
{'NOUN': {'NOUN': 0.2642562484833778, '.': 0.2400942905674767, 'ADJ': 0.012132977432661975, 'ADP': 0.17679481401878877, 'NUM': 0.00946372239747634, 'VERB': 0.14698235518424793, 'ADV': 0.016986168405726764, 'PRT': 0.04392137830623635, 'DET': 0.013068949977467327, 'CONJ': 0.04260408361354734, 'PRON': 0.004679862724026762, 'X': 0.029015148888965923}, '.': {'NUM': 0.11684331503000894, 'VERB': 0.12808070489081855, 'DET': 0.1422551398288852, 'NOUN': 0.18733239688417827, 'ADP': 0.07214915081087983, '.': 0.09909334695441195, 'PRON': 0.062061039458562124, 'ADV': 0.05261141616651768, 'ADJ': 0.045843442727620996, 'CONJ': 0.061550249010343505, 'PRT': 0.0029370450772570555, 'X': 0.0292427531605159}}

Emission probabilities:
{'NOUN': {'Pierre': 3.464163231371462e-05, 'Vinken': 6.928326462742924e-05, 'years': 0.00

In [ ]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
import nltk
from collections import defaultdict
import numpy as np

def calculate_hmm_probabilities(tagged_sentences):
    """Calculates HMM probabilities from tagged sentences."""
    initial_counts = defaultdict(int)
    transition_counts = defaultdict(lambda: defaultdict(int))
    emission_counts = defaultdict(lambda: defaultdict(int))
    state_counts = defaultdict(int)

    for sentence in tagged_sentences:
        if not sentence:
            continue
        initial_counts[sentence[0][1]] += 1
        state_counts[sentence[0][1]] += 1

        for i in range(len(sentence) - 1):
            current_tag = sentence[i][1]
            next_tag = sentence[i + 1][1]
            word = sentence[i][0]
            transition_counts[current_tag][next_tag] += 1
            emission_counts[current_tag][word] += 1
            state_counts[next_tag] += 1
        emission_counts[sentence[-1][1]][sentence[-1][0]] += 1

    states = set(state_counts.keys())
    total_initial = sum(initial_counts.values())
    initial_probabilities = {tag: count / total_initial for tag, count in initial_counts.items()}

    transition_probabilities = {}
    for prev_tag, next_tags in transition_counts.items():
        total_transitions = sum(next_tags.values())
        transition_probabilities[prev_tag] = {tag: count / total_transitions for tag, count in next_tags.items()}
        for state in states:
            if state not in transition_probabilities[prev_tag]:
                transition_probabilities[prev_tag][state] = 0

    emission_probabilities = {}
    for tag, words in emission_counts.items():
        total_emissions = sum(words.values())
        emission_probabilities[tag] = {word: count / total_emissions for word, count in words.items()}

    return initial_probabilities, transition_probabilities, emission_probabilities, states

def sentence_probability_hmm(observations, states, initial_probabilities, transition_probabilities, emission_probabilities):
    """Calculates the probability of an observed sentence given an HMM."""
    num_observations = len(observations)
    num_states = len(states)
    forward_probabilities = np.zeros((num_states, num_observations))

    for i, state in enumerate(states):
        if observations[0] in emission_probabilities[state]:
            forward_probabilities[i, 0] = initial_probabilities[state] * emission_probabilities[state][observations[0]]
        else:
            forward_probabilities[i, 0] = 0

    for t in range(1, num_observations):
        for i, current_state in enumerate(states):
            prob_sum = 0
            for j, previous_state in enumerate(states):
                if observations[t] in emission_probabilities[current_state]:
                    prob_sum += forward_probabilities[j, t - 1] * transition_probabilities[previous_state][current_state] * emission_probabilities[current_state][observations[t]]
                else:
                  prob_sum += 0
            forward_probabilities[i, t] = prob_sum

    sentence_prob = np.sum(forward_probabilities[:, num_observations - 1])
    return sentence_prob

def get_nltk_tagged_sentences(sentences):
    """Tags sentences using NLTK's pos_tag."""
    tagged_sentences = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        tagged_sentences.append(nltk.pos_tag(words))
    return tagged_sentences

# Example usage
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

training_sentences = [
    "the cat sat on the mat",
    "a cat ran in the house",
    "the dog sat on a mat",
    "a dog ran in the garden"
]

tagged_sentences = get_nltk_tagged_sentences(training_sentences)
initial_probs, transition_probs, emission_probs, states = calculate_hmm_probabilities(tagged_sentences)

test_sentence = nltk.word_tokenize("the cat ran")

sentence_probability = sentence_probability_hmm(test_sentence, states, initial_probs, transition_probs, emission_probs)
print(f"Probability of the test sentence: {sentence_probability}")

Probability of the test sentence: 0.008333333333333333


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
import nltk
from collections import defaultdict
import numpy as np

def calculate_hmm_probabilities(tagged_sentences):
    """Calculates HMM probabilities from tagged sentences."""
    init = defaultdict(int)
    trans = defaultdict(lambda: defaultdict(int))
    emit = defaultdict(lambda: defaultdict(int))
    state_count = defaultdict(int)

    for s in tagged_sentences:
        if not s: continue
        init[s[0][1]] += 1
        state_count[s[0][1]] += 1
        for i in range(len(s)-1):
            trans[s[i][1]][s[i+1][1]] += 1
            emit[s[i][1]][s[i][0]] += 1
            state_count[s[i+1][1]] += 1
        emit[s[-1][1]][s[-1][0]] += 1

    states = set(state_count.keys())
    total = sum(init.values())
    init_prob = {t: c/total for t, c in init.items()}

    # Initialize all possible transitions with 0 probability
    trans_prob = {state: {s: 0 for s in states} for state in states}
    for p, next_states in trans.items():
        total_trans = sum(next_states.values())
        for t, c in next_states.items():
            trans_prob[p][t] = c/total_trans if total_trans > 0 else 0

    # Initialize emission probs for all states, even if empty
    emit_prob = {state: {} for state in states}
    for t, words in emit.items():
        total_words = sum(words.values())
        emit_prob[t] = {w: c/total_words for w, c in words.items()}

    return init_prob, trans_prob, emit_prob, states

def sentence_probability_hmm(obs, states, init_prob, trans_prob, emit_prob):
    """Calculates sentence probability using HMM."""
    n_obs, n_states = len(obs), len(states)
    fwd = np.zeros((n_states, n_obs))

    for i, s in enumerate(states):
        # Use .get() for init_prob with default 0 for states not seen in initial position
        fwd[i, 0] = init_prob.get(s, 0) * emit_prob[s].get(obs[0], 0)

    for t in range(1, n_obs):
        for i, curr in enumerate(states):
            fwd[i, t] = sum(fwd[j, t-1] * trans_prob[prev][curr] *
                           emit_prob[curr].get(obs[t], 0) for j, prev in enumerate(states))

    return np.sum(fwd[:, n_obs-1])

# Example usage
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

train = ["the cat sat on the mat", "a cat ran in the house",
         "the dog sat on a mat", "a dog ran in the garden"]

tagged = [nltk.pos_tag(nltk.word_tokenize(s)) for s in train]
init_p, trans_p, emit_p, states = calculate_hmm_probabilities(tagged)
test = nltk.word_tokenize("the cat sat in the house")
prob = sentence_probability_hmm(test, states, init_p, trans_p, emit_p)
print(f"Probability of the test sentence: {prob}")
print(trans_p)
print(emit_p)

Probability of the test sentence: 0.0013020833333333333
{'VBD': {'VBD': 0, 'NN': 0, 'DT': 0, 'IN': 1.0}, 'NN': {'VBD': 0.3333333333333333, 'NN': 0.3333333333333333, 'DT': 0, 'IN': 0.3333333333333333}, 'DT': {'VBD': 0, 'NN': 1.0, 'DT': 0, 'IN': 0}, 'IN': {'VBD': 0, 'NN': 0, 'DT': 1.0, 'IN': 0}}
{'VBD': {'sat': 1.0}, 'NN': {'cat': 0.2, 'mat': 0.2, 'ran': 0.2, 'house': 0.1, 'dog': 0.2, 'garden': 0.1}, 'DT': {'the': 0.625, 'a': 0.375}, 'IN': {'on': 0.5, 'in': 0.5}}
